国会会議録と同じように、csvを機械学習にかける。
前回は使用比率によって色々変えたけど、とりあえず一つのパターンでやってみる。
可能であれば関数化もする。

In [4]:
import sys;sys.path.append('./src')
import sqlite3
import re
from control_db import *
from NLP import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import re 
from collections import Counter
import urllib.request as urllib
from bs4 import BeautifulSoup
import sqlite3
import time
from contextlib import closing
import sklearn
import seaborn as sns
# import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [5]:
ldf=pd.read_csv('./1121low_mor_goiword.csv')
hdf=pd.read_csv('./1121high_mor_goiword.csv')


In [7]:
LABEL_NAME=['高評価','低評価']
CLASIFFIER={'RandomForest':RandomForestClassifier(random_state=1),
                    'NB':MultinomialNB()}
DF_LIST={'高評価':hdf,
                '低評価':ldf}

USE_PERCENTS=[0.001,0.0004,0.0001,0.00001]

In [ ]:
def preproDF2(df,PERCENT):
    fdf=pd.DataFrame()
    fdf['sum_word']=fdf.sum(axis=1).astype(int)
    fdf=fdf.sort_values('sum_word',ascending=False)
    fdf['kouseihi']=fdf['sum_word']/fdf['sum_word'].sum()
    fdf['ruisekihi']=fdf['sum_word'].cumsum()/fdf['sum_word'].sum()

    fdf=fdf.reset_index(drop=True)
    fdf=fdf[fdf['kouseihi']>PERCENT]
    
    wordcouttdf=fdf[['Unnamed: 0','sum_word','kouseihi','ruisekihi']]
    fdf.drop(columns=['Unnamed: 0','sum_word','kouseihi','ruisekihi'],inplace=True)

    fdf=fdf.T
    wordcouttdf=wordcouttdf.T

    fdf['label']=1
    fdf.loc[fdf.index.str.contains(word+'減'),'label']=0
    
    target=fdf['label']
    fdf=fdf.drop(columns=['label'])
#     fdf=fdf.reset_index(drop=True)
    return(fdf,target,wordcouttdf)